# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where T
    lattice=T(a) * I(3)  # lattice is a cube of ``a`` Bohrs
    He = ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))
    atoms = [He => [[1/2; 1/2; 1/2]]]  # Helium at the center of the box

    model = model_DFT(lattice, atoms, [:lda_x, :lda_c_vwn];
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...), tol=tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            Eₙ-Eₙ₋₁     ρout-ρin   α      Diag
---   ---------------   ---------   --------   ----   ----
  1   -2.770849429020         NaN   2.99e-01   0.80    8.0
  2   -2.772143819061   -1.29e-03   4.79e-02   0.80    1.0
  3   -2.772170243153   -2.64e-05   3.80e-03   0.80    2.0
  4   -2.772170722573   -4.79e-07   7.18e-05   0.80    2.0
  5   -2.772170723000   -4.26e-10   1.19e-05   0.80    2.0
n     Energy            Eₙ-Eₙ₋₁     ρout-ρin   α      Diag
---   ---------------   ---------   --------   ----   ----
  1   -2.770721129585         NaN   2.97e-01   0.80    8.0
  2   -2.772030512986   -1.31e-03   5.03e-02   0.80    1.0
  3   -2.772083185384   -5.27e-05   1.49e-03   0.80    2.0
  4   -2.772083413240   -2.28e-07   3.02e-04   0.80    2.0
  5   -2.772083417752   -4.51e-09   3.73e-05   0.80    2.0


1.773821682992295

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            Eₙ-Eₙ₋₁     ρout-ρin   α      Diag
---   ---------------   ---------   --------   ----   ----
  1   -2.770743675950         NaN   2.99e-01   0.80    9.0
  2   -2.772058993264   -1.32e-03   4.77e-02   0.80    1.0
  3   -2.772082696600   -2.37e-05   4.13e-03   0.80    2.0
  4   -2.772083417615   -7.21e-07   1.09e-04   0.80    2.0
  5   -2.772083417798   -1.84e-10   1.50e-05   0.80    1.0

Polarizability via ForwardDiff:       1.7725420814911585
Polarizability via finite difference: 1.773821682992295
